In [11]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output

data = pd.read_csv('Datasets/classified_data1.csv',error_bad_lines=False, index_col=False, dtype='unicode', nrows=100000)
#data = data.query("state='Alabama'")
#ata["date"] = pd.to_datetime(data["date"], format="%Y-%m-%d")
data.sort_values("likeCount", inplace = True)
data["likeCount"] = pd.to_numeric(data["likeCount"])
app = JupyterDash(__name__)

minimum,maximum = 0, 1000



In [12]:


app.layout = html.Div([
        dcc.Graph(id="like-over-time"),
    html.P("Like Slider"),
    dcc.RangeSlider(
        id="range-slider",
        min=minimum, max = maximum, step = 1,
        marks={minimum: str(minimum), maximum: str(maximum)},
        value=[minimum,maximum]
    ),
    
    
    ]   
)

@app.callback(
    Output("like-over-time", "figure"), 
    [Input("range-slider", "value")])
def update_bar_chart(slider_range):
    low, high = slider_range
    print(low,high)
    mask = (data['likeCount'] > low) & (data['likeCount'] < high)
    print(mask)
    fig = px.scatter(
        data[mask], x="date", y="likeCount")
    return fig


In [13]:
app.run_server(mode='inline', port=1002)

0 1000
47071    False
49955    False
49956    False
49958    False
49959    False
49962    False
49963    False
49964    False
49965    False
49966    False
49969    False
49970    False
49972    False
49973    False
49975    False
49976    False
49977    False
49980    False
49981    False
49983    False
49985    False
49986    False
49987    False
49990    False
49999    False
50000    False
49950    False
49945    False
49944    False
49943    False
         ...  
99970    False
99971    False
99972    False
99973    False
99974    False
99975    False
99976    False
99977    False
99978    False
99979    False
99980    False
99981    False
99982    False
99983    False
99984    False
99985    False
99986    False
99987    False
99988    False
99989    False
99990    False
99991    False
99992    False
99993    False
99994    False
99995    False
99996    False
99997    False
99998    False
99999    False
Name: likeCount, Length: 100000, dtype: bool
871 1000
47071    False
49955    

ValueError: Mixing dicts with non-Series may lead to ambiguous ordering.